# Quickstart

This section explains how to use MQC3 through the example code.

## Three representations of quantum circuits

Before diving into the example code, we first introduce the three different representations of quantum circuits in MQC3: **circuit representation**, **graph representation**, and **machinery representation**.
To execute an optical quantum computing using MQC3, you need to create a machinery representation of the quantum circuit.
If you start with a circuit representation, MQC3 will automatically convert it to a graph representation, and then further transform it into a machinery representation.

| Representation           | Brief Description |
| -----------------------  | ----------------- |
| Circuit representation   | A standard continuos-variable quantum circuit consisting of modes and operations performed on modes. See [Constructing Circuit Representation](circuit_repr.ipynb) for details. |
| Graph representation     | An intermediate representation between the circuit representation and the machinery representation. See [Constructing Graph Representation](graph_repr.ipynb) for details. |
| Machinery representation | The final data format used for executing the optical quantum computer of MQC3. See [Constructing Machinery Representation](machinery_repr.ipynb) and [Theory](theory.md) for details. |

### Example: Circuit Representation

The image below shows an example of a circuit representation.
This circuit consists of two modes, mode `0` and mode `1`, and performs five operations in sequence:

1. Rotate mode `0` by $\frac{\pi}{4}$
2. Displace the $x$-quadrature of mode `0` by $1$ and the $p$-quadrature by $-1$
3. Perform a controlled-Z operation between mode `0` and mode `1`
4. Measure mode `0` at a measurement angle of $0$
5. Measure mode `1` at a measurement angle of $\frac{\pi}{2}$

See [Gates](gates.md) for the definition of these operations.

![circuit](_images/circuit_repr_sample_circuit.svg)

### Example: Graph Representation

The image below illustrates the graph representation corresponding to the above circuit representation.
The graph representation can be visualized in two dimensions, as shown below.
The twelve large circles are called **macronodes**, and each macronode contains four small circles, which are called **micronodes**.

* A "step" consists of a group of vertically aligned macronodes.
* The number of macronodes arranged vertically is the number of local macronodes per step.
* The number of macronodes arranged horizontally is the number of steps.

In this example:

* The number of local macronodes per step is `3`.
* The number of steps is `4`.

The blue micronode represents mode `0`, which undergoes transformations as it passes through four macronodes.
The red micronode corresponds to mode `1`, which is processed through three macronodes.
Both modes are initialized at the first macronode and measured at the last macronode.
Thus, this graph representation is equivalent to the circuit representation shown above.

![graph](_images/quickstart_graph.png)

## Creating a circuit

Let's create the following quantum circuit and run it on the optical quantum computer.
This circuit performs rotation and measurement on mode `0`.

![quickstart_example](_images/quickstart_example.svg)

First, import the necessary packages.

In [ ]:
from math import pi

import matplotlib.pyplot as plt
import numpy as np

from mqc3.circuit import CircuitRepr
from mqc3.circuit.ops import intrinsic
from mqc3.client import MQC3Client
from mqc3.execute import execute

You can create a circuit using {py:class}`~mqc3.circuit.CircuitRepr`.
A mode can be created using {py:meth}`~mqc3.circuit.CircuitRepr.Q`.
Operations are created using {py:class}`~mqc3.circuit.ops.intrinsic.PhaseRotation` and {py:class}`~mqc3.circuit.ops.intrinsic.Measurement`.

In [ ]:
circuit = CircuitRepr("example")
circuit.Q(0) | intrinsic.PhaseRotation(pi / 4)
circuit.Q(0) | intrinsic.Measurement(pi / 4)

circuit

## Creating a client

To use the optical quantum computer, you need to create an execution client by instantiating {py:class}`~mqc3.client.MQC3Client`.
You also need to set up your API token to send the circuit to the optical quantum computer.

In [ ]:
client = MQC3Client()
client.backend = "emulator"
client.n_shots = 1000
# client.token = "YOUR_API_TOKEN"

## Executing the circuit

You can execute the circuit using {py:func}`~mqc3.execute.execute`.
This function takes a circuit and a client as arguments and returns the execution result.

In [ ]:
result = execute(circuit, client)

## Checking the execution result

The execution result is returned as an instance of {py:class}`~mqc3.execute.ExecutionResult`.
You can print the result as shown below.

In [ ]:
from pprint import pprint

pprint(result)

You can get the execution result of the 0th shot as shown below.

In [ ]:
pprint(result[0])

If you want to check the transformed graph representation and its result, refer to {py:attr}`~mqc3.client.MQC3ClientResult.compiled_graph` and {py:attr}`~mqc3.client.MQC3ClientResult.graph_result`.

In [ ]:
from mqc3.client import MQC3ClientResult

assert isinstance(result.client_result, MQC3ClientResult)
graph = result.client_result.compiled_graph
graph_result = result.client_result.graph_result

You can plot the graph using {py:func}`~mqc3.graph.visualize.make_figure` to verify the result of the transformation into the graph representation.

In [ ]:
from mqc3.graph.visualize import make_figure

assert graph is not None
make_figure(graph, ignore_wiring=True);

In [ ]:
assert graph_result is not None
print(graph_result[1])  # Measurement results of the first shot.

If you want to check the transformed machinery representation and its result, refer to {py:attr}`~mqc3.client.MQC3ClientResult.compiled_machinery` and {py:attr}`~mqc3.client.MQC3ClientResult.machinery_result`.

In [ ]:
assert isinstance(result.client_result, MQC3ClientResult)
machine = result.client_result.compiled_machinery
machine_result = result.client_result.machinery_result

In [ ]:
assert machine is not None
pprint(machine.get_homodyne_angle(0))  # Measurement angles at macronode index 0.
pprint(machine.get_homodyne_angle(1))  # Measurement angles at macronode index 1.

In [ ]:
assert machine_result is not None
print(machine_result[1])  # Measurement results of the first shot.

## Checking the rotation result

As an example of a slightly more complex code, the following example demonstrates how to validate the previously mentioned phase rotation.
This time, the mode after the rotation is measured at various angles ($\theta=0,\frac{\pi}{8},\frac{\pi}{4},\dots,\pi$), and the variance of the measured values is calculated for each angle.

![quickstart_multi_param](_images/quickstart_multi_param.svg)

In [ ]:
def construct_circuit(theta: float) -> CircuitRepr:
    circuit = CircuitRepr(f"theta={theta}")
    circuit.Q(0) | intrinsic.PhaseRotation(pi / 4)
    circuit.Q(0) | intrinsic.Measurement(theta)

    return circuit


# Define measurement angles: 0, pi/8, pi/4, ..., pi.
angle_list = np.linspace(0, pi, 9)

var_list = []

for theta in angle_list:
    circuit = construct_circuit(theta)
    result = execute(circuit, client)

    # Collect the measured values for mode 0.
    mode0_values = [smv[0].value for smv in result]

    # Calculate and store the variance.
    var_list.append(np.var(mode0_values))

print(var_list)

### Plotting the variance

The mode is initially in an $x$-squeezed state (squeezing angle=$\frac{\pi}{2}$).
After the $\frac{\pi}{4}$ rotation, the state acquires a squeezing angle of $\frac{3\pi}{4}$.
Therefore, when measured in the direction of $\frac{3\pi}{4}$, the variance is minimized, whereas in the direction of $\frac{\pi}{4}$, the variance is maximized.

In [ ]:
angle_labels = [
    r"$0$",
    r"$\frac{\pi}{8}$",
    r"$\frac{\pi}{4}$",
    r"$\frac{3\pi}{8}$",
    r"$\frac{\pi}{2}$",
    r"$\frac{5\pi}{8}$",
    r"$\frac{3\pi}{4}$",
    r"$\frac{7\pi}{8}$",
    r"$\pi$",
]

plt.figure(figsize=(10, 6))
plt.plot(angle_list, var_list, "ro-", label="Mode 0")

plt.xlabel(r"Measurement angle")
plt.ylabel("Variance")
plt.xticks(angle_list, angle_labels)
plt.legend()
plt.show()